In [2]:
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /home/nathaniel/anaconda3/lib/python3.7/site-packages (0.22.2.post1)


Load packages

In [3]:
import sklearn
import pandas as pd
import numpy as np
from math import pi

import matplotlib.pyplot as plt
import seaborn as sns

#from IPython.display import display, Markdown


In [32]:
df_games = pd.read_csv("../nba-games/games.csv",parse_dates=["GAME_DATE_EST"],infer_datetime_format=True)
df_games = df_games.drop_duplicates()\
    .drop(columns=["TEAM_ID_home", "TEAM_ID_away", "GAME_STATUS_TEXT"], axis=1)\
    .sort_values("GAME_DATE_EST", ascending=False)

df_games.shape

(23195, 18)

In [33]:
df_games = df_games.drop(columns=["TEAM_ID_home", "TEAM_ID_away", "GAME_STATUS_TEXT"], axis=1)
df_games.shape

KeyError: "['TEAM_ID_home' 'TEAM_ID_away' 'GAME_STATUS_TEXT'] not found in axis"

In [34]:
df = df_games[df_games.SEASON == 2018]
df.shape

(1378, 18)

In [35]:
df["HOME_TEAM_WIN_LAST_10_GAMES"] = 0
df["VISITOR_TEAM_WIN_LAST_10_GAMES"] = 0
df.shape

/home/nathaniel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/nathaniel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(1378, 20)

In [59]:
df_preprocessed = pd.DataFrame()

df_preprocessed = df[["HOME_TEAM_ID", "VISITOR_TEAM_ID"]].copy()

df_preprocessed.sample()

,HOME_TEAM_ID,VISITOR_TEAM_ID
16912,1610612745,1610612743


In [60]:
def last_10_games(team_id):
    team_df = df[(df.HOME_TEAM_ID == team_id) | (df.VISITOR_TEAM_ID == team_id)].head(10)
    balance = 0
    for idx, row in team_df.iterrows():
        if row.HOME_TEAM_ID == team_id:
            balance += row.HOME_TEAM_WINS
        else:
            if row.HOME_TEAM_WINS == 0:
                balance += 1
    return balance

df = df.sort_values(by='GAME_DATE_EST')

for idx, row in df.iterrows():
   df.at[idx, "HOME_TEAM_WIN_LAST_10_GAMES"] = last_10_games(row ["HOME_TEAM_ID"])
   df.at[idx, "VISITOR_TEAM_WIN_LAST_10_GAMES"] = last_10_games(row ["VISITOR_TEAM_ID"])


In [56]:
list(df.columns)

['GAME_DATE_EST',
 'GAME_ID',
 'HOME_TEAM_ID',
 'VISITOR_TEAM_ID',
 'SEASON',
 'PTS_home',
 'FG_PCT_home',
 'FT_PCT_home',
 'FG3_PCT_home',
 'AST_home',
 'REB_home',
 'PTS_away',
 'FG_PCT_away',
 'FT_PCT_away',
 'FG3_PCT_away',
 'AST_away',
 'REB_away',
 'HOME_TEAM_WINS',
 'HOME_TEAM_WIN_LAST_10_GAMES',
 'VISITOR_TEAM_WIN_LAST_10_GAMES']

In [61]:
#print(df_preprocessed.shape)
#print(df.shape)
#
#df[["VISITOR_TEAM_WIN_LAST_10_GAMES", "HOME_TEAM_WIN_LAST_10_GAMES"]]


df_preprocessed = pd.concat(
    [df_preprocessed,
     df[["VISITOR_TEAM_WIN_LAST_10_GAMES", "HOME_TEAM_WIN_LAST_10_GAMES"]]]
    , axis=1
)
df_preprocessed.head(5)

,HOME_TEAM_ID,VISITOR_TEAM_ID,VISITOR_TEAM_WIN_LAST_10_GAMES,HOME_TEAM_WIN_LAST_10_GAMES
16196,1610612744,1610612761,9,5
16197,1610612761,1610612744,5,9
16198,1610612744,1610612761,9,5
16199,1610612744,1610612761,9,5
16200,1610612761,1610612744,5,9
